In [ ]:
import os
import uuid
from dotenv import load_dotenv, find_dotenv
import getpass
from sqlalchemy import create_engine, Column, Integer, String, Text, DateTime, ForeignKey, func
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import Mapped, mapped_column, DeclarativeBase, sessionmaker, relationship
from sqlalchemy.dialects import postgresql
from sqlalchemy.dialects.postgresql import UUID
from datetime import datetime, timezone
import pytz
import enum
from dateutil import parser as date_parser

from pydantic import BaseModel, Field
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain_deepseek import ChatDeepSeek
from langchain_openai import ChatOpenAI, AzureChatOpenAI
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.tools import tool
from langchain.agents import create_openai_functions_agent, create_react_agent, Tool, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.tools.render import format_tool_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain.schema.agent import AgentFinish
from langchain.schema.runnable import RunnablePassthrough

load_dotenv(find_dotenv(), override=True)

# --- Configuration ---
DB_USER = os.getenv("DB_USER", "postgres")
DB_PASSWORD = os.getenv("DB_PASSWORD", "postgres")
DB_HOST = os.getenv("DB_HOST", "localhost")
DB_PORT = os.getenv("DB_PORT", "1234")
DB_NAME = os.getenv("DB_NAME", "postgres") 

DATABASE_URL = f"postgresql://{DB_USER}:{DB_PASSWORD}@{DB_HOST}:{DB_PORT}/{DB_NAME}"

Base = declarative_base()
engine = create_engine(DATABASE_URL)
Session = sessionmaker(bind=engine, expire_on_commit=False)

class TaskPriority(enum.Enum):
    LOW = "low"
    MEDIUM = "medium"
    HIGH = "high"

class TaskStatus(enum.Enum):
    ACTIVE = "active"
    COMPLETED = "completed"

class User(Base):
    __tablename__ = 'users' 

    user_id = Column(UUID(as_uuid=True), primary_key=True, default=uuid.uuid4, unique=True, nullable=False)
    username = Column(String(50), unique=True, nullable=False)
    email = Column(String(100), unique=True, nullable=False)
    bio = Column(Text, nullable=True)
    
    # Store user's preferred timezone as IANA timezone identifier
    timezone = Column(String(50), nullable=False, default='UTC')
    
    created_at = Column(DateTime(timezone=True), default=func.now())
    updated_at = Column(DateTime(timezone=True), default=func.now(), onupdate=func.now())

    tasks = relationship("Task", back_populates="user", cascade="all, delete-orphan")

    def __repr__(self):
        return f"<User(id={self.user_id}, username='{self.username}', timezone='{self.timezone}')>"

class Task(Base):
    __tablename__ = 'tasks'

    task_id = Column(UUID(as_uuid=True), primary_key=True, default=uuid.uuid4, unique=True, nullable=False)
    task_name = Column(String(100), nullable=False)
    task_description = Column(Text, nullable=True)

    # Foreign Key
    user_id = Column(Integer, ForeignKey('users.user_id'), nullable=False)
 
    created_at = Column(DateTime(timezone=True), default=func.now())
    updated_at = Column(DateTime(timezone=True), default=func.now(), onupdate=func.now())
    start_time_utc = Column(DateTime(timezone=True), nullable=False)
    end_time_utc = Column(DateTime(timezone=True), nullable=False)

    priority: Mapped[TaskPriority] = mapped_column(
        postgresql.ENUM(
            TaskPriority, 
            name="task_priority_enum",
            create_type=True 
        ),
        nullable=True,
    )

    status: Mapped[TaskPriority] = mapped_column(
        postgresql.ENUM(
            TaskStatus, 
            name="task_status_enum",
            create_type=True
        ),
        default=TaskStatus.ACTIVE  
    )
    
    # Store the original timezone for reference
    original_timezone = Column(String(50), nullable=True)
    
    # RFC 5545 RRULE string for repeats
    recurrence_rule = Column(Text, nullable=True) 

    user = relationship("User", back_populates="tasks")

    def __repr__(self):
        return f"<Task(id={self.task_id}, name='{self.task_name}', start={self.start_time_utc}, end={self.end_time_utc}, user_id={self.user_id})>"

def get_valid_timezones():
    """Return a list of all valid timezone strings."""
    return pytz.all_timezones

def validate_timezone(tz_string):
    """Validate if a timezone string is valid."""
    return tz_string in pytz.all_timezones

def get_common_timezones():

    return {
        'US/Eastern': 'America/New_York',
        'US/Central': 'America/Chicago',
        'US/Mountain': 'America/Denver',
        'US/Pacific': 'America/Los_Angeles',
        'Europe/London': 'Europe/London',
        'Europe/Paris': 'Europe/Paris',
        'Europe/Berlin': 'Europe/Berlin',
        'Asia/Tokyo': 'Asia/Tokyo',
        'Asia/Shanghai': 'Asia/Shanghai',
        'Asia/Kolkata': 'Asia/Kolkata',
        'Australia/Sydney': 'Australia/Sydney',
        'UTC': 'UTC'
    }

def local_to_utc(local_dt, timezone_str):
    """Convert local datetime to UTC."""
    if not isinstance(local_dt, datetime):
        raise ValueError("local_dt must be a datetime object")
    
    # Get timezone object
    local_tz = pytz.timezone(timezone_str)
    
    if local_dt.tzinfo is None:
        local_dt = local_tz.localize(local_dt)
    
    # Convert to UTC
    return local_dt.astimezone(pytz.UTC)

def utc_to_local(utc_dt, timezone_str):
    """Convert UTC datetime to local timezone."""
    if not isinstance(utc_dt, datetime):
        raise ValueError("utc_dt must be a datetime object")

    if utc_dt.tzinfo is None:
        utc_dt = pytz.UTC.localize(utc_dt)
    
    # Get target timezone
    local_tz = pytz.timezone(timezone_str)
    
    # Convert to local timezone
    return utc_dt.astimezone(local_tz)

def add_user(username, email, timezone_str, bio=None):
    """
    Add a new user.
    Args:
        username (str): Unique username
        email (str): Unique email address
        timezone_str (str): IANA timezone identifier (e.g., 'America/New_York')
        bio (str, optional): User biography
    Returns:
        User: The created user object or None if failed
    """
    # Validate timezone
    if not validate_timezone(timezone_str):
        print(f"Error: Invalid timezone '{timezone_str}'")
        print(f"Use one of: {', '.join(get_common_timezones().values())}")
        return None
    
    session = Session()
    try:
        new_user = User(
            username=username,
            email=email,
            bio=bio,
            timezone=timezone_str
        )
        session.add(new_user)
        session.commit()
        print(f"User '{username}' added successfully with timezone: {timezone_str}")
        return new_user
    except Exception as e:
        session.rollback()
        print(f"Error adding user: {e}")
        return None
    finally:
        session.close()

class AddTaskInput(BaseModel):
    user_id: int = Field(..., description="ID of the user")
    task_name: str = Field(..., description="Name of the task")
    start_time_local: datetime = Field(..., description="Task start time")
    end_time_local: datetime = Field(..., description="Task end time")
    task_description: str = Field(None, description="Task description")

@tool(args_schema=AddTaskInput)
def add_task(user_id : int, 
             task_name : str, 
             start_time_local : datetime, 
             end_time_local : datetime, 
             task_description: str = None) -> str:
    """
    Add a new task to the database.
    Returns: Success or error message indicating the result of the operation.
    """
    session = Session()
    try:
        user = session.query(User).filter_by(user_id=user_id).first()
        if not user:
            return "User not found"
        
        if user.timezone != 'UTC':
            start_utc = local_to_utc(start_time_local, user.timezone)
            end_utc = local_to_utc(end_time_local, user.timezone)

        original_tz = user.timezone

        new_task = Task(
            task_name=task_name,
            task_description=task_description,
            user_id=user_id,
            start_time_utc=start_utc,
            end_time_utc=end_utc,
            original_timezone=original_tz,
        )
        session.add(new_task)
        session.commit()
        return f"Task '{task_name}' added successfully"
    except Exception as e:
        session.rollback()
        return f"Error adding task: {e}"
    finally:
        session.close()

class GetUserTasksInput(BaseModel):
    user_id: int = Field(..., description="ID of the user whose tasks to retrieve")
    start_date: datetime = Field(None, description="Filter tasks starting after this date")
    end_date: datetime = Field(None, description="Filter tasks ending before this date")

@tool(args_schema=GetUserTasksInput)
def get_user_tasks(user_id : int, 
                   start_date : datetime = None, 
                   end_date : datetime = None) -> str:
    """
    Get user's tasks.
    Returns: String of user's tasks.
    """
    session = Session()
    try:
        user = session.query(User).filter_by(user_id=user_id).first()
        if not user:
            return "User not found"
        
        query = session.query(Task).filter_by(user_id=user_id)
        
        # Apply date filters if provided
        if start_date:
            if user.timezone != 'UTC':
                start_utc = local_to_utc(start_date, user.timezone)
            else:
                start_utc = start_date
            query = query.filter(Task.start_time_utc >= start_utc)
        
        if end_date:
            if user.timezone != 'UTC':
                end_utc = local_to_utc(end_date, user.timezone)
            else:
                end_utc = end_date
            query = query.filter(Task.end_time_utc <= end_utc)
        
        tasks = query.all()
        
        user_tasks = "User tasks:\n\n"
        for task in range(len(tasks)):
            task_dict = f"""
'task_id': {tasks[task].task_id}
'task_name': {tasks[task].task_name}
'task_description': {tasks[task].task_description}
'start_time_local': {utc_to_local(tasks[task].start_time_utc, user.timezone)}
'end_time_local': {utc_to_local(tasks[task].end_time_utc, user.timezone)}
'timezone': {user.timezone}
            """
            # {
            #     'task_id': tasks[task].task_id,
            #     'task_name': tasks[task].task_name,
            #     'task_description': tasks[task].task_description,
            #     'start_time_local': utc_to_local(tasks[task].start_time_utc, user.timezone),
            #     'end_time_local': utc_to_local(tasks[task].end_time_utc, user.timezone),
            #     'timezone': user.timezone,
            #     # 'recurrence_rule': tasks[task].recurrence_rule
            # }
            user_tasks += f"{task_dict}\n"

        return user_tasks
    finally:
        session.close()

class UpdateTaskInput(BaseModel):
    task_id: int = Field(..., description="ID of the task to update")
    task_name: str = Field(None, description="New name for the task")
    task_description: str = Field(None, description="New description for the task")
    start_time_local: datetime = Field(None, description="New start time")
    end_time_local: datetime = Field(None, description="New end time")

@tool(args_schema=UpdateTaskInput)
def update_task(task_id : int, 
                task_name : str =None, 
                task_description : str =None,
                start_time_local : datetime =None, 
                end_time_local : datetime =None) -> str:
    """
    Update an existing task.
    Returns: Success or error message indicating the result of the operation.
    """
    session = Session()
    try:                    
        task = session.query(Task).filter_by(task_id=task_id).first()
        if not task:
            return "Task not found"
        
        # Get user to determine timezone
        user = session.query(User).filter_by(user_id=task.user_id).first()
        if not user:
            return "User not found"
        
        # Update fields
        if task_name:
            task.task_name = task_name

        if task_description:
            task.task_description = task_description
        
        if start_time_local:
            if user.timezone != 'UTC':
                start_utc = local_to_utc(start_time_local, user.timezone)
            else:
                start_utc = start_time_local
            task.start_time_utc = start_utc
        
        if end_time_local:
            if user.timezone != 'UTC':
                end_utc = local_to_utc(end_time_local, user.timezone)
            else:
                end_utc = end_time_local
            task.end_time_utc = end_utc

        task.original_timezone = user.timezone # problem if the user change the timezone after task creation 

        # if priority:
        #     if isinstance(priority, TaskPriority):
        #         task.priority = priority
        #     else:
        #         print(f"Error: Invalid priority value '{priority}'")
        
        # if status:
        #     if isinstance(status, TaskStatus):
        #         task.status = status
        #     else:
        #         print(f"Error: Invalid status value '{status}'")
        
        # if recurrence_rule:
        #     task.recurrence_rule = recurrence_rule
        
        session.commit()
        return f"Task '{task.task_name}' updated successfully"
    except Exception as e:
        session.rollback()
        return f"Error updating task: {e}"
    finally:
        session.close()
        
def example_timezone_usage():
    """Demonstrate timezone-aware operations."""
    print("\n=== Timezone-Aware Example ===\n")
    
    # Create tables
    Base.metadata.create_all(engine)
    
    # Add users from different timezones
    user_ny = add_user("ny_user", "ny@example.com", "America/New_York", "User in New York")
    user_london = add_user("london_user", "london@example.com", "Europe/London", "User in London")
    user_tokyo = add_user("tokyo_user", "tokyo@example.com", "Asia/Tokyo", "User in Tokyo")
    
    if user_ny and user_london and user_tokyo:
        # Schedule a meeting at 3 PM New York time
        meeting_time_ny = datetime(2024, 1, 15, 15, 0)  # 3 PM NY time
        
        # Add the same meeting for all users (it will be stored in UTC)
        for user in [user_ny, user_london, user_tokyo]:
            add_task(
                user_id=user.user_id,
                task_name="Global Team Meeting",
                start_time_local=meeting_time_ny,
                end_time_local=datetime(2024, 1, 15, 16, 0),
                task_description="Quarterly review meeting",
                use_user_timezone=False  # We're providing NY time for everyone
            )
        
        # Now let's see how each user sees the meeting
        print("\n=== How each user sees the meeting ===")
        for user in [user_ny, user_london, user_tokyo]:
            tasks = get_user_tasks(user.user_id)
            if tasks:
                task = tasks[0]
                print(f"\n{user.username} ({user.timezone}):")
                print(f"  Meeting time: {task['start_time_local'].strftime('%Y-%m-%d %H:%M %Z')}")

def display_timezone_selector():
    """Example of how to display timezones for user selection."""
    print("\n=== Common Timezones for User Selection ===")
    for display_name, tz_id in get_common_timezones().items():
        # Get current time in that timezone
        now_utc = datetime.now(pytz.UTC)
        local_time = now_utc.astimezone(pytz.timezone(tz_id))
        offset = local_time.strftime('%z')
        current_time = local_time.strftime('%H:%M')
        print(f"{display_name:20} (UTC{offset[:3]}:{offset[3:]}) - Current time: {current_time}")

# if __name__ == "__main__":
    
#     example_timezone_usage()
#     display_timezone_selector()
#     print("\nScript finished.")

/tmp/ipykernel_3402/1447359293.py:38: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


In [ ]:
example_timezone_usage()

In [ ]:
display_timezone_selector()

In [ ]:
update_task(
    task_id=1,
    task_name="Updated Global Team Meeting",
    start_time_local=datetime(2024, 1, 15, 14, 0),  # Change to 2 PM NY time
    end_time_local=datetime(2024, 1, 15, 15, 0),
    priority=TaskPriority.HIGH,
    status=TaskStatus.ACTIVE,
    use_user_timezone=True
)
print("\n=== Updated Task ===")
tasks = get_user_tasks(user_id=1)
for task in tasks:
    print(f"Task ID: {task['task_id']}, Name: {task['task_name']}, "
          f"Start: {task['start_time_local'].strftime('%Y-%m-%d %H:%M %Z')}, "
          f"End: {task['end_time_local'].strftime('%Y-%m-%d %H:%M %Z')}, "
          f"Recurrence: {task['recurrence_rule']}")
print("\nScript finished.")

In [ ]:
utc_now = datetime.now(pytz.utc)

new_york_timezone = pytz.timezone('America/New_York')
ny_time = utc_now.astimezone(new_york_timezone)
user_timezone = pytz.timezone("America/New_York")
current_time = ny_time.strftime('%Y-%m-%d %I:%M:%S %p')
current_day = ny_time.strftime("%A")
user_id = 1

tools = [get_user_tasks, add_task, update_task]

functions = [format_tool_to_openai_function(f) for f in tools]

model = AzureChatOpenAI(
    azure_deployment="gpt-4",
    api_version="2024-12-01-preview",
    temperature=0
).bind(functions=functions)

system_message = f"""
You are a highly organized, meticulous, and helpful AI assistant dedicated to managing personal tasks.
Your primary goal is to help the user effectively organize their tasks.

You have access to the following tools to interact with the task management system:
1. **`get_user_tasks(user_id: int, start_date: Optional[datetime] = None, end_date: Optional[datetime] = None)`**:
    * **Description**: Retrieves the user's tasks, optionally filtered by start and end dates.
    * **Parameters**:
        * `user_id` (int): The unique identifier of the user whose tasks are being retrieved.
        * `start_date` (Optional[datetime]): If provided, only tasks starting after this date will be returned. You must parse this from the user's natural language input, considering the current date and time context.
        * `end_date` (Optional[datetime]): If provided, only tasks ending before this date will be returned. You must parse this from the user's natural language input, considering the current date and time context.
    * **Returns**: A string of user's tasks.

2.  **`add_task(user_id: int, task_name: str, start_time_local: datetime, end_time_local: datetime, task_description: Optional[str] = None)`**:
    * **Description**: Adds a new task to the user's personal task list.
    * **Parameters**:
        * `user_id` (int): The unique identifier of the user who owns this task.
        * `task_name` (str): A clear, concise name or title for the task.
        * `start_time_local` (datetime): The *local* start date and time of the task. You must parse this from the user's natural language input, considering the current date and time context.
        * `end_time_local` (datetime): The *local* end date and time of the task. You must parse this from the user's natural language input, considering the current date and time context.
        * `task_description` (Optional[str]): An optional, more detailed description of the task.
    * **Returns**: A confirmation message indicating the task was successfully added, or an error message if the operation failed.

3.  **`update_task(task_id: int, task_name: str, start_time_local: datetime, end_time_local: datetime, task_description: Optional[str] = None)`**:
    * **Description**: Updates an existing task in the user's personal task list.
    * **Parameters**:
        * `task_id` (int): The unique identifier of the task to be updated.
        * `task_name` (Optional[str]): A clear, concise name or title for the task.
        * `start_time_local` (Optional[datetime]): The start date and time of the task. You must parse this from the user's natural language input, considering the current date and time context.
        * `end_time_local` (Optional[datetime]): The end date and time of the task. You must parse this from the user's natural language input, considering the current date and time context.
        * `task_description` (Optional[str]): An optional, more detailed description of the task.
    * **Returns**: A confirmation message indicating the task was successfully updated, or an error message if the operation failed.
     
---

**CRITICAL PROTOCOL FOR TASK MODIFICATIONS (Creation or Update):**

To ensure user control and prevent unintended changes, you **MUST ALWAYS** follow this explicit approval process for **ANY** request that involves adding or modifying tasks:
1.  **Understand Current State**: Call `get_user_tasks()` to retrieve the user's current task list and then ask the user for their request. This gives you the context of what tasks currently exist and what the user is asking for.
2.  **Formulate Proposal**: Based on the user's request and the user current task list, clearly formulate how the created or updated task **would look** after the requested changes. This includes:
    * For new tasks: Show the new task details to the user.
    * For updated tasks: Show the task with its modified fields.
3.  **Present for Approval**: Present this *proposed future task list* (or the specific proposed changes in context) to the user in a clear, easy-to-read format (e.g., a bulleted list, a formatted table).
4.  **Seek Confirmation**: Explicitly ask the user for confirmation of these proposed changes. For example: "Here is how your task list would look after these changes. Does this look correct, or would you like to make any adjustments?" or "Please review the proposed tasks below and confirm if I should proceed."
5.  **Execute ONLY on Approval**: **ONLY after the user explicitly confirms and approves the proposed changes** (e.g., by saying "Yes," "Go ahead," "Confirm," "Looks good"), you are then authorized to call the `create_task` or `update_task` tool(s) accordingly.
6.  **Handle Ambiguity**: If the user's request is ambiguous or lacks necessary information for a task modification, ask clarifying questions before proposing any changes.

**CRITICAL TASK UPDATE PROTOCOL:**

Before Modifying Any Task, ALWAYS:
1. Verify Ownership and Authenticity
   - MANDATORY: Cross-check the provided task ID with the user's existing task list
   - CONFIRM the task ID belongs to the specific user requesting the modification
   - PREVENT unauthorized or incorrect task updates

Verification Steps:
- Retrieve current user task list
- Match provided task ID exactly
- If task ID cannot be verified, IMMEDIATELY halt the update process

Update Workflow:
- Task ID Verification ✓ → Proceed with Update
- Task ID Verification ✗ → Reject Update, Report Error

Error Handling:
- If task ID is invalid or not found, return a clear error message
- Do not make ANY modifications without explicit, verified authorization

Security Principle: 
"Verify First, Update Second" - ZERO exceptions to this rule.

Always prioritize user clarity and confirmation. Be concise and empathetic in your responses.

--- 

Here is some information about the user's current context:
- **Current Time:** {current_time}
- **Current Day:** {current_day}
- **User ID:** {user_id}
- **User Timezone:** {user_timezone}

Please use this information to provide contextually relevant and timely responses. 
For example, if asked about scheduling, consider their current day and time.
"""

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)

agent_chain = RunnablePassthrough.assign(
    agent_scratchpad= lambda x: format_to_openai_functions(x["intermediate_steps"])
) | prompt | model | OpenAIFunctionsAgentOutputParser()

def run_agent(user_input):
    intermediate_steps = []
    while True:
        result = agent_chain.invoke({
            "input": user_input, 
            "agent_scratchpad": intermediate_steps
        })
        if isinstance(result, AgentFinish):
            return result
        tool = {
            "get_user_tasks": get_user_tasks,
            "add_task": add_task,
            "update_task": update_task
        }[result.tool]
        observation = tool.run(result.tool_input)
        intermediate_steps.append((result, observation))

memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")

agent_executor = AgentExecutor(agent=agent_chain, tools=tools, verbose=True, memory=memory)

In [8]:
user_input = """
Hello! 
"""

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...
Hello! How can I assist you today with your tasks?

> Finished chain.
Task Planner: Hello! How can I assist you today with your tasks?


In [9]:
user_input = "What time is it right now?"

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...
The current time is 2:37 AM on Thursday, June 26, 2025, in the America/New_York timezone. How can I assist you further today?

> Finished chain.
Task Planner: The current time is 2:37 AM on Thursday, June 26, 2025, in the America/New_York timezone. How can I assist you further today?


In [10]:
user_input = """
what is my current task ?
"""

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")




> Entering new AgentExecutor chain...

Invoking: `get_user_tasks` with `{'user_id': 1}`


User tasks:


            'task_id': 19
            'task_name': Prepare presentation for team meeting
            'task_description': None
            'start_time_local': 2025-07-15 11:00:00-04:00
            'end_time_local': 2025-07-19 16:00:00-04:00
            'timezone': America/New_York
            

            'task_id': 20
            'task_name': Create a diagram for the upcoming meeting
            'task_description': Create a diagram for the upcoming meeting.
            'start_time_local': 2025-07-25 10:00:00-04:00
            'end_time_local': 2025-07-25 11:00:00-04:00
            'timezone': America/New_York
            
Currently, you have the following tasks scheduled:

1. **Prepare presentation for team meeting**
   - **Start Time:** July 15, 2025, 11:00 AM
   - **End Time:** July 19, 2025, 4:00 PM
   - **Description:** Not specified

2. **Create a diagram for the upcoming mee

In [11]:
user_input = "Help me add a task to my list. I need to schedule a meeting with the team tomorrow at 10 AM for 1 hour. The meeting is about the quarterly review. Please add it to my task list."

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...
Before adding the task, let's review the details:

- **Task Name:** Team Meeting for Quarterly Review
- **Start Time:** June 27, 2025, 10:00 AM
- **End Time:** June 27, 2025, 11:00 AM
- **Description:** Meeting about the quarterly review.

Here is how your task list would look after adding this new task:

1. **Prepare presentation for team meeting**
   - **Start Time:** July 15, 2025, 11:00 AM
   - **End Time:** July 19, 2025, 4:00 PM
   - **Description:** Not specified

2. **Create a diagram for the upcoming meeting**
   - **Start Time:** July 25, 2025, 10:00 AM
   - **End Time:** July 25, 2025, 11:00 AM
   - **Description:** Create a diagram for the upcoming meeting.

3. **Team Meeting for Quarterly Review**
   - **Start Time:** June 27, 2025, 10:00 AM
   - **End Time:** June 27, 2025, 11:00 AM
   - **Description:** Meeting about the quarterly review.

Please review the proposed task above and confirm if I should proceed with adding it to your 

In [12]:
user_input = "Yes that looks good. Please go ahead and add the task."

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...

Invoking: `add_task` with `{'user_id': 1, 'task_name': 'Team Meeting for Quarterly Review', 'start_time_local': '2025-06-27T10:00:00', 'end_time_local': '2025-06-27T11:00:00', 'task_description': 'Meeting about the quarterly review.'}`


Task 'Team Meeting for Quarterly Review' added successfully for user 'ny_user'The task "Team Meeting for Quarterly Review" has been successfully added to your list. Is there anything else you would like to do or any other assistance you need?

> Finished chain.
Task Planner: The task "Team Meeting for Quarterly Review" has been successfully added to your list. Is there anything else you would like to do or any other assistance you need?


In [14]:
user_input = f"Help me update the presentation task. I want to change the start time to August 15th, 2025, at 11 AM New York time, and the end time to August 19th, 2025, at 4 PM New York time."

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...
Let's review the updated details for the "Prepare presentation for team meeting" task:

- **New Start Time:** August 15, 2025, 11:00 AM
- **New End Time:** August 19, 2025, 4:00 PM

Here is how your task list would look after this update:

1. **Prepare presentation for team meeting**
   - **Start Time:** August 15, 2025, 11:00 AM
   - **End Time:** August 19, 2025, 4:00 PM
   - **Description:** Not specified

2. **Create a diagram for the upcoming meeting**
   - **Start Time:** July 25, 2025, 10:00 AM
   - **End Time:** July 25, 2025, 11:00 AM
   - **Description:** Create a diagram for the upcoming meeting.

3. **Team Meeting for Quarterly Review**
   - **Start Time:** June 27, 2025, 10:00 AM
   - **End Time:** June 27, 2025, 11:00 AM
   - **Description:** Meeting about the quarterly review.

Please review the proposed changes above and confirm if I should proceed with updating the task.

> Finished chain.
Task Planner: Let's review the updated d

In [15]:
user_input = "yes, that looks good. Please go ahead and update the task."

try:
    response = agent_executor.invoke({"input": user_input})
    print(f"Task Planner: {response['output']}")
except Exception as e:
    print(f"Task Planner: I encountered an error: {str(e)}")



> Entering new AgentExecutor chain...

Invoking: `update_task` with `{'task_id': 1, 'task_name': 'Prepare presentation for team meeting', 'start_time_local': '2025-08-15T11:00:00-04:00', 'end_time_local': '2025-08-19T16:00:00-04:00'}`


Task not found
Invoking: `get_user_tasks` with `{'user_id': 1}`
responded: It seems there was an issue finding the task to update. Let me ensure the task ID is correct and try updating it again. Please hold on for a moment.

User tasks:


            'task_id': 19
            'task_name': Prepare presentation for team meeting
            'task_description': None
            'start_time_local': 2025-07-15 11:00:00-04:00
            'end_time_local': 2025-07-19 16:00:00-04:00
            'timezone': America/New_York
            

            'task_id': 20
            'task_name': Create a diagram for the upcoming meeting
            'task_description': Create a diagram for the upcoming meeting.
            'start_time_local': 2025-07-25 10:00:00-04:00
 

In [16]:
memory.load_memory_variables({})

{'chat_history': [HumanMessage(content='\nHello! \n', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Hello! How can I assist you today with your tasks?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='What time is it right now?', additional_kwargs={}, response_metadata={}),
  AIMessage(content='The current time is 2:37 AM on Thursday, June 26, 2025, in the America/New_York timezone. How can I assist you further today?', additional_kwargs={}, response_metadata={}),
  HumanMessage(content='\nwhat is my current task ?\n', additional_kwargs={}, response_metadata={}),
  AIMessage(content='Currently, you have the following tasks scheduled:\n\n1. **Prepare presentation for team meeting**\n   - **Start Time:** July 15, 2025, 11:00 AM\n   - **End Time:** July 19, 2025, 4:00 PM\n   - **Description:** Not specified\n\n2. **Create a diagram for the upcoming meeting**\n   - **Start Time:** July 25, 2025, 10:00 AM\n   - **End Time:** July 25, 2025, 11:00 AM\n